In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import io
from PIL import Image, ImageOps
import math
from scipy.spatial import distance_matrix
from utils import cv2_display_image, getColors, colorToLetter, fixColors
from vision import process_color_images, imageToColors
from rembg import remove
import twophase.solver  as sv


#3 face image, and try to register a cube
# depth mask before background removal?
# register a box to an image?
# If you know camera properties you could label them.
# segmentation the background,


loading conj_twist table...
loading conj_ud_edges table...
loading flipslice sym-tables...
loading corner sym-tables...
loading move_twist table...
loading move_flip table...
loading move_slice_sorted table...
loading move_u_edges table...
loading move_d_edges table...
loading move_ud_edges table...
loading move_corners table...
loading phase1_prun table...
loading phase2_prun table...
loading phase2_cornsliceprun table...
loading phase2_edgemerge table...


In [2]:
image_path = "images/scramble_3/blue.jpg"
color_names = imageToColors(image_path)
color_names

i, j 0 0
1.0 177.0 135.0
i, j 0 1
110.0 132.0 77.0
i, j 0 2
48.0 138.0 109.0
i, j 1 0
10.0 216.0 233.0
i, j 1 1
110.0 120.0 86.0
i, j 1 2
28.0 215.0 153.0
i, j 2 0
11.0 222.0 241.0
i, j 2 1
21.0 36.0 199.0
i, j 2 2
26.0 220.0 178.0


[['red', 'blue', 'green'],
 ['orange', 'blue', 'yellow'],
 ['orange', 'white', 'yellow']]

In [3]:
directory_path = "images/scramble_3/"
faces = process_color_images(directory_path)
faces = fixColors(faces)


RUBIKS COLORS {'yellow': array([ 13., 146., 164.]), 'blue': array([85., 59., 47.]), 'orange': array([ 36.,  93., 199.]), 'green': array([ 39., 114.,  80.]), 'white': array([139., 164., 179.]), 'red': array([ 27.,  36., 101.])}
i, j 0 0
21.0 28.0 183.0
i, j 0 1
3.0 159.0 104.0
i, j 0 2
111.0 113.0 82.0
i, j 1 0
109.0 151.0 99.0
i, j 1 1
27.0 234.0 164.0
i, j 1 2
9.0 206.0 200.0
i, j 2 0
2.0 198.0 139.0
i, j 2 1
3.0 204.0 138.0
i, j 2 2
3.0 187.0 117.0
i, j 0 0
1.0 177.0 135.0
i, j 0 1
110.0 132.0 77.0
i, j 0 2
48.0 138.0 109.0
i, j 1 0
10.0 216.0 233.0
i, j 1 1
110.0 120.0 86.0
i, j 1 2
28.0 215.0 153.0
i, j 2 0
11.0 222.0 241.0
i, j 2 1
21.0 36.0 199.0
i, j 2 2
26.0 220.0 178.0
i, j 0 0
110.0 114.0 89.0
i, j 0 1
16.0 62.0 181.0
i, j 0 2
19.0 53.0 175.0
i, j 1 0
17.0 53.0 202.0
i, j 1 1
10.0 208.0 199.0
i, j 1 2
43.0 167.0 108.0
i, j 2 0
11.0 228.0 237.0
i, j 2 1
42.0 189.0 122.0
i, j 2 2
43.0 173.0 104.0
i, j 0 0
26.0 212.0 177.0
i, j 0 1
26.0 218.0 167.0
i, j 0 2
26.0 209.0 168.0
i, j

In [4]:
faces

{'yellow': [['white', 'red', 'blue'],
  ['blue', 'yellow', 'orange'],
  ['red', 'red', 'red']],
 'blue': [['red', 'blue', 'green'],
  ['orange', 'blue', 'yellow'],
  ['orange', 'white', 'yellow']],
 'orange': [['blue', 'white', 'white'],
  ['white', 'orange', 'green'],
  ['orange', 'green', 'green']],
 'green': [['yellow', 'yellow', 'yellow'],
  ['red', 'green', 'red'],
  ['white', 'green', 'yellow']],
 'white': [['orange', 'white', 'green'],
  ['orange', 'white', 'yellow'],
  ['orange', 'blue', 'white']],
 'red': [['red', 'orange', 'green'],
  ['blue', 'red', 'green'],
  ['blue', 'yellow', 'blue']]}

In [5]:
order = ["yellow", "orange", "green", "white", "red", "blue"]
convert = colorToLetter()
CUBE_STRING = ""
for color in order:
    for i in range(len(faces[color])):
        for j in range(len(faces[color][0])):
            CUBE_STRING += (convert[faces[color][i][j]])
CUBE_STRING

'DLBBURLLLBDDDRFRFFUUULFLDFURDFRDURBDLRFBLFBUBLBFRBURDU'

In [6]:
solution = sv.solve(CUBE_STRING, 19, 2)

In [16]:
solution = solution.replace("3", "'")
solution = solution.replace("1","")
solution


"L D F2 U' F2 B' U' D F' D2 R B U L2 U' B2 U' B2 U' (9f)"